In [2]:
import tweepy
import pandas as pd
import re
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [3]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAFhhtgEAAAAAeA6bZaP2s42L8WfivYQX6Yw7QU0%3DOnnbjVEvCEC7NheuWE7OrKlFNpxlSAIf9OJAoFgtkT13brJ4Ua'

In [4]:

base_url = 'https://api.twitter.com/2/'

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json'
}

In [ ]:
def fetch_tweets(user_handle, count=10):
    url = f'{base_url}users/by/username/{user_handle}/tweets'
    params = {
        'max_results': count,
        'tweet.fields': 'created_at,text'
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        tweet_list = [[tweet['created_at'], tweet['text']] for tweet in data['data']]
        return pd.DataFrame(tweet_list, columns=['Date', 'Tweet'])
    else:
        print(f'Error fetching tweets: {response.status_code}, {response.text}')
        return pd.DataFrame(columns=['Date', 'Tweet'])



# chagne user handle, can add user input if needed


user_handle = 'localdreamer_z'




df_tweets = fetch_tweets(user_handle, count=10)
print(df_tweets.head())

if df_tweets.empty:
    print(f'No tweets found for user @{user_handle}.')
else:
    print(df_tweets.head())


In [6]:

def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # remove URLs
    tweet = re.sub(r'@\w+', '', tweet)  # remove mentions
    tweet = re.sub(r'#\w+', '', tweet)  # remove hashtags
    tweet = re.sub(r'[^A-Za-z0-9\s]', '', tweet)  # remove special characters
    tweet = tweet.lower()  # convert to lowercase
    return tweet

df_tweets['Cleaned_Tweet'] = df_tweets['Tweet'].apply(clean_tweet)


In [ ]:


sentiment_pipeline = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment')

df_tweets['Sentiment'] = df_tweets['Cleaned_Tweet'].apply(lambda x: sentiment_pipeline(x)[0]['label'])


In [8]:

def categorize_sentiment(label):
    if label == 'LABEL_0':
        return 'Negative'
    elif label == 'LABEL_1':
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:

df_tweets['Sentiment_Category'] = df_tweets['Sentiment'].apply(categorize_sentiment)

plt.figure(figsize=(10, 6))
sns.countplot(data=df_tweets, x='Sentiment_Category', order=['Negative', 'Neutral', 'Positive'])
plt.title('Sentiment Analysis of Tweets')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()
